In [1]:

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys
import time

import numpy as np
import tensorflow as tf

from tensorflow.python.platform import app
from delf import feature_io
from delf.python.detect_to_retrieve import dataset


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [2]:
IS_QUERY = False

In [3]:
# Extensions.
_DELF_EXTENSION = '.delf'

# Default DELF dimensionality.
_DELF_DIM = 128

# Pace to report log when collecting features.
_STATUS_CHECK_ITERATIONS = 100

OUTPUT_PATH = 'output/cluster/index'
FEATURE_PATH = 'output/feature'

if IS_QUERY : 
    OUTPUT_PATH = 'output/cluster/test'
    FEATURE_PATH = 'output/test'
else :
    OUTPUT_PATH = 'output/cluster/index'
    FEATURE_PATH = 'output/feature'


In [4]:
if os.path.exists(OUTPUT_PATH):
    raise RuntimeError(
            'output_cluster_dir = %s already exists. This may indicate that a '
            'previous run already wrote checkpoints in this directory, which would '
            'lead to incorrect training. Please re-run this script by specifying an'
            ' inexisting directory.' % OUTPUT_PATH)
else : 
    os.makedirs(OUTPUT_PATH)

    

In [5]:
# Read list of index images from dataset file.
print('Reading list of index images from dataset file...')

if IS_QUERY : 
    image_list = os.listdir("image_test")
else :
    image_list = os.listdir("image_index")

num_images = len(image_list)
print('done! Found %d images' % num_images)

Reading list of index images from dataset file...
done! Found 7618 images


In [6]:
# Loop over list of index images and collect DELF features.
features_for_clustering = []
start = time.clock()
print('Starting to collect features from index images...')
for i in range(num_images):
    if i > 0 and i % _STATUS_CHECK_ITERATIONS == 0:
        elapsed = (time.clock() - start)
        print('Processing index image %d out of %d, last %d '
                    'images took %f seconds' %
                    (i, num_images, _STATUS_CHECK_ITERATIONS, elapsed))
        start = time.clock()

    features_filename = image_list[i] + _DELF_EXTENSION
    features_fullpath = os.path.join(FEATURE_PATH, features_filename)
    _, _, features, _, _ = feature_io.ReadFromFile(features_fullpath)
    if features.size != 0:
        assert features.shape[1] == _DELF_DIM
    for feature in features:
        features_for_clustering.append(feature)

features_for_clustering = np.array(features_for_clustering, dtype=np.float32)
print('All features were loaded! There are %d features, each with %d '
            'dimensions' %
            (features_for_clustering.shape[0], features_for_clustering.shape[1]))

Starting to collect features from index images...
Instructions for updating:
Use tf.gfile.GFile.


C:\Users\kohry\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\kohry\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys
C:\Users\kohry\anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  # This is added back by InteractiveShellApp.init_path()


Processing index image 100 out of 7618, last 100 images took 0.703755 seconds
Processing index image 200 out of 7618, last 100 images took 0.757947 seconds
Processing index image 300 out of 7618, last 100 images took 0.786481 seconds
Processing index image 400 out of 7618, last 100 images took 0.784635 seconds
Processing index image 500 out of 7618, last 100 images took 0.692960 seconds
Processing index image 600 out of 7618, last 100 images took 0.786027 seconds
Processing index image 700 out of 7618, last 100 images took 0.826134 seconds
Processing index image 800 out of 7618, last 100 images took 0.761833 seconds
Processing index image 900 out of 7618, last 100 images took 0.797485 seconds
Processing index image 1000 out of 7618, last 100 images took 0.734378 seconds
Processing index image 1100 out of 7618, last 100 images took 0.781817 seconds
Processing index image 1200 out of 7618, last 100 images took 0.784039 seconds
Processing index image 1300 out of 7618, last 100 images took

In [7]:
class _IteratorInitHook(tf.train.SessionRunHook):
    """Hook to initialize data iterator after session is created."""

    def __init__(self):
        super(_IteratorInitHook, self).__init__()
        self.iterator_initializer_fn = None

    def after_create_session(self, session, coord):
        """Initialize the iterator after the session has been created."""
        del coord
        self.iterator_initializer_fn(session)

In [8]:
# Run K-means clustering.
def _get_input_fn():
    """Helper function to create input function and hook for training.

    Returns:
        input_fn: Input function for k-means Estimator training.
        init_hook: Hook used to load data during training.
    """
    init_hook = _IteratorInitHook()

    def _input_fn():
        """Produces tf.data.Dataset object for k-means training.

        Returns:
            Tensor with the data for training.
        """
        features_placeholder = tf.placeholder(tf.float32, features_for_clustering.shape)
        delf_dataset = tf.data.Dataset.from_tensor_slices((features_placeholder))
        delf_dataset = delf_dataset.shuffle(1000).batch(features_for_clustering.shape[0])
        iterator = delf_dataset.make_initializable_iterator()

        def _initializer_fn(sess):
            """Initialize dataset iterator, feed in the data."""
            sess.run(
                    iterator.initializer,
                    feed_dict={features_placeholder: features_for_clustering})

        init_hook.iterator_initializer_fn = _initializer_fn
        return iterator.get_next()

    return _input_fn, init_hook

In [9]:
input_fn, init_hook = _get_input_fn()

# kmeans = tf.estimator.experimental.KMeans(
#         num_clusters=1000, # cluster숫자
#         model_dir=OUTPUT_PATH,
#         use_mini_batch=False,
# )

kmeans = tf.contrib.factorization.KMeansClustering(
    num_clusters= 1000,
    use_mini_batch=False,
    model_dir=OUTPUT_PATH
)

iteration_count = 50

print('Starting K-means clustering...')
start = time.clock()
for i in range(iteration_count): #임의의 숫자로 클러스러이
    kmeans.train(input_fn, hooks=[init_hook])
    average_sum_squared_error = kmeans.evaluate(
            input_fn, hooks=[init_hook])['score'] / features_for_clustering.shape[0]
    elapsed = (time.clock() - start)
    print('K-means iteration %d (out of %d) took %f seconds, '
                'average-sum-of-squares: %f' %
                (i, iteration_count, elapsed, average_sum_squared_error))
    start = time.clock()

print('K-means clustering finished!')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'output/cluster/index', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001F0CB553C18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Starting K-means clustering...
Instructions for updating:
Colocations handled automatically by placer.
INFO:tensorflow:Calling model_

C:\Users\kohry\anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into output/cluster/index\model.ckpt.
INFO:tensorflow:Saving checkpoints for 1 into output/cluster/index\model.ckpt.
INFO:tensorflow:Loss for final step: None.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-28T07:21:31Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from output/cluster/index\model.ckpt-1
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-28-07:21:35
INFO:tensorflow:Saving dict for global step 1: global_step = 1, loss = 10

C:\Users\kohry\anaconda3\lib\site-packages\ipykernel_launcher.py:23: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\kohry\anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/cluster/index\model.ckpt-1
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into output/cluster/index\model.ckpt.
INFO:tensorflow:loss = 1087979.2, step = 2
INFO:tensorflow:Saving checkpoints for 3 into output/cluster/index\model.ckpt.
INFO:tensorflow:Loss for final step: 1087979.2.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-28T07:21:42Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/cluster/index\model.ckpt-3
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-28-07:21:47
INFO:tensorflow:Saving dict for global step 3: global_step = 3, loss = 717160.1, score = 717160.1

...

INFO:tensorflow:Loss for final step: 675954.44.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-28T07:25:12Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/cluster/index\model.ckpt-41
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-28-07:25:17
INFO:tensorflow:Saving dict for global step 41: global_step = 41, loss = 675779.25, score = 675779.25
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 41: output/cluster/index\model.ckpt-41
K-means iteration 20 (out of 50) took 10.867525 seconds, average-sum-of-squares: 0.641175
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/cluster/index\model.ckpt-41
INFO:tensorflow:Running local_init_o

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/cluster/index\model.ckpt-53
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 53 into output/cluster/index\model.ckpt.
INFO:tensorflow:loss = 675024.4, step = 54
INFO:tensorflow:Saving checkpoints for 55 into output/cluster/index\model.ckpt.
INFO:tensorflow:Loss for final step: 675024.4.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-28T07:26:32Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/cluster/index\model.ckpt-55
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-28-07:26:37
INFO:tensorflow:Saving dict for global step 55: global_step = 55, loss = 674931.4

INFO:tensorflow:Restoring parameters from output/cluster/index\model.ckpt-67
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-28-07:27:44
INFO:tensorflow:Saving dict for global step 67: global_step = 67, loss = 674487.56, score = 674487.56
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 67: output/cluster/index\model.ckpt-67
K-means iteration 33 (out of 50) took 11.223478 seconds, average-sum-of-squares: 0.639949
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/cluster/index\model.ckpt-67
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 67 into output/cluster/index\model.ckpt.
INFO:tensorflow:loss = 674487.56, step = 67
INFO:tensorflow:Saving checkpoints for 69 into 

INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 79 into output/cluster/index\model.ckpt.
INFO:tensorflow:loss = 674190.3, step = 80
INFO:tensorflow:Saving checkpoints for 81 into output/cluster/index\model.ckpt.
INFO:tensorflow:Loss for final step: 674190.3.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-28T07:28:57Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/cluster/index\model.ckpt-81
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-28-07:29:02
INFO:tensorflow:Saving dict for global step 81: global_step = 81, loss = 674149.56, score = 674149.56
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 81: output/cluster/index\model.ckpt-81
K-means iteration 40 (out of 50) took 11.298694 seconds, average-sum-of-

INFO:tensorflow:Saving dict for global step 93: global_step = 93, loss = 673925.5, score = 673925.5
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 93: output/cluster/index\model.ckpt-93
K-means iteration 46 (out of 50) took 11.030517 seconds, average-sum-of-squares: 0.639416
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output/cluster/index\model.ckpt-93
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 93 into output/cluster/index\model.ckpt.
INFO:tensorflow:loss = 673925.5, step = 94
INFO:tensorflow:Saving checkpoints for 95 into output/cluster/index\model.ckpt.
INFO:tensorflow:Loss for final step: 673925.5.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-28T07:30:16Z
INFO:tens